In [ ]:
#CALLBACKS
from keras.layers import BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Input, Dropout

data = pd.read_csv('/content/ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True)
data.set_index('Date', inplace=True)
data.head()

df1 = pd.DataFrame(data['Close'])
data = df1.values
train_test_split = int(np.ceil(0.65 * len(data)))
train_data = data[:train_test_split]
test_data = data[train_test_split:]
train_data.shape, test_data.shape

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])


x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))


model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=15, verbose=1)
early_stopping = EarlyStopping(monitor='loss', patience=15, verbose=1)

# Training
model.fit(x_train, y_train, epochs=200, batch_size=5, callbacks=[reduce_lr, early_stopping])

y_train = scaler.inverse_transform(y_train)
y_train_predict = model.predict(x_train)
y_train_predict = scaler.inverse_transform(y_train_predict)

gru_r2 = r2_score(y_train, y_train_predict)
gru_mse = mean_squared_error(y_train, y_train_predict)
gru_mae = mean_absolute_error(y_train, y_train_predict)

print('MSE:', gru_mse) 
print('MAE:', gru_mae) 
print('R2 Score:', gru_r2)


In [ ]:
#FINE TUNING
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from scipy.stats import uniform

# Load data
data = pd.read_csv('ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True)

# Preprocess data
df1 = pd.DataFrame(data['Close'])
data = df1.values
train_test_split = int(np.ceil(0.65 * len(data)))
train_data = data[:train_test_split]
test_data = data[train_test_split:]

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Prepare the data for LSTM/GRU
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))

# Define a function to create the GRU model
def create_gru_model(units=50, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(GRU(units=units, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    model.add(GRU(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Wrap the model for use in RandomizedSearchCV
model = KerasRegressor(build_fn=create_gru_model, verbose=0)

# Define the parameter grid to search
param_dist = {
    'model__units': [50, 100, 150, 250],
    'model__dropout_rate': uniform(0.1, 0.4),  # Random values between 0.1 and 0.5
    'batch_size': [8, 16, 32, 64],
    'epochs': [50, 100, 200],
    'optimizer': ['adam', 'rmsprop']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=10, cv=3, verbose=2, random_state=42)

# Fit the RandomizedSearchCV
random_search.fit(x_train, y_train)

# Print the best parameters and score
print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)

# Predict using the best model
best_model = random_search.best_estimator_

y_train_predict = best_model.predict(x_train)
y_train_predict = scaler.inverse_transform(y_train_predict.reshape(-1, 1))
y_train = scaler.inverse_transform(y_train)

# Evaluate the model
gru_r2 = r2_score(y_train, y_train_predict)
gru_mse = mean_squared_error(y_train, y_train_predict)
gru_mae = mean_absolute_error(y_train, y_train_predict)

print('MSE:', gru_mse)
print('MAE:', gru_mae)
print('R2 Score:', gru_r2)